# PART 1: DATA LOADING & PREPROCESSING
## Setup, Data Loading, and Calculated Fields

This section handles all data operations, imports, and derived field calculations needed for the analysis.

In [1]:
# === IMPORTS & SETUP ===
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# === AESTHETIC CONFIGURATION ===
# FUTURISTIC GLOWING COLOR SCHEME (matching dashboard)
COLORS = {
    'primary': '#8B5FBF',      # Purple gradient
    'secondary': '#22D3EE',    # Cyan blue  
    'accent': '#F97316',       # Orange accent
    'success': '#10B981',      # Green
    'warning': '#F59E0B',      # Amber
    'danger': '#EF4444',       # Red
    'pink': '#EC4899',         # Pink for cards
    'dark_bg': '#0A0A1A',      # Ultra dark background
    'card_bg': '#1A1A2E',      # Dark card background with glow
    'sidebar_bg': '#16213E',   # Glowing sidebar
    'light': '#FFFFFF',        # White
    'text': '#E2E8F0',         # Light gray text
    'text_dim': '#94A3B8',     # Dimmed text
    'neon_blue': '#00F5FF',    # Neon blue glow
    'neon_purple': '#BF40BF',  # Neon purple glow
    'neon_green': '#39FF14',   # Neon green glow
    'glow_shadow': 'rgba(34, 211, 238, 0.4)',  # Glowing shadow
    'neutral': '#6C7B7F'       # Cool gray
}

# Professional color palettes using dashboard colors
citizenship_colors = [COLORS['primary'], COLORS['secondary'], COLORS['accent'], COLORS['pink']]
funding_colors = [COLORS['neon_blue'], COLORS['neon_purple'], COLORS['neon_green'], COLORS['accent'], COLORS['pink']]
gender_colors = [COLORS['primary'], COLORS['secondary']]

# Global styling with futuristic dark theme
template_config = {
    'layout': {
        'font': {'family': 'Orbitron, monospace', 'size': 12, 'color': COLORS['light']},
        'paper_bgcolor': COLORS['dark_bg'],
        'plot_bgcolor': 'rgba(0,0,0,0)',
        'margin': {'l': 80, 'r': 120, 't': 150, 'b': 80},
        'hovermode': 'closest',
        'showlegend': True,
        'legend': {
            'orientation': 'h',
            'yanchor': 'bottom',
            'y': -0.2,
            'xanchor': 'center',
            'x': 0.5,
            'bgcolor': 'rgba(255,255,255,0.8)',
            'bordercolor': COLORS['neutral'],
            'borderwidth': 1
        }
    }
}

print("📊 Imports and aesthetic configuration completed!")

📊 Imports and aesthetic configuration completed!


In [2]:
# === DATA LOADING & PREPROCESSING ===
print("📊 Loading and preprocessing data...")

# Load datasets
try:
    profiles = pd.read_excel("Cleaned_Student_Profiles.xlsx")
    results = pd.read_excel("Cleaned_Semester_Results.xlsx")
    courses = pd.read_excel("Cleaned_Course_Codes.xlsx")
    print("✅ Data loaded successfully")
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    # Create mock data for demonstration
    print("🔄 Creating mock data for demonstration...")
    np.random.seed(42)
    n_students = 500
    
    profiles = pd.DataFrame({
        'STUDENT ID': [f'STU{i:05d}' for i in range(1, n_students + 1)],
        'GENDER': np.random.choice(['Male', 'Female'], n_students),
        'CITIZENSHIP': np.random.choice(['Citizen', 'PR', 'International'], n_students, p=[0.6, 0.25, 0.15]),
        'HIGHEST QUALIFICATION': np.random.choice(['Diploma', 'Bachelor', 'Master', 'PhD'], n_students, p=[0.3, 0.4, 0.25, 0.05]),
        'COURSE FUNDING': np.random.choice(['Self-Funded', 'Scholarship', 'Sponsored', 'Loan'], n_students),
        'COMMENCEMENT DATE': pd.date_range('2020-01-01', '2023-12-31', periods=n_students),
        'COMPLETION DATE': pd.date_range('2022-01-01', '2026-12-31', periods=n_students),
        'FULL-TIME OR PART-TIME': np.random.choice(['Full-Time', 'Part-Time'], n_students, p=[0.7, 0.3])
    })
    
    # Generate results data
    periods = ['2020 Sem 1', '2020 Sem 2', '2021 Sem 1', '2021 Sem 2', '2022 Sem 1', '2022 Sem 2', '2023 Sem 1', '2023 Sem 2']
    n_results = n_students * 3  # Average 3 semesters per student
    results = pd.DataFrame({
        'STUDENT ID': np.random.choice(profiles['STUDENT ID'], n_results),
        'PERIOD': np.random.choice(periods, n_results),
        'GPA': np.random.beta(2, 1, n_results) * 3 + 1  # GPA between 1-4
    })
    results['GPA'] = results['GPA'].round(2)

# Parse dates
date_columns = ['COMMENCEMENT DATE', 'COMPLETION DATE']
for col in date_columns:
    if col in profiles.columns:
        profiles[col] = pd.to_datetime(profiles[col], errors='coerce')

print("✅ Data loading completed!")

📊 Loading and preprocessing data...
✅ Data loaded successfully
✅ Data loading completed!


In [3]:
# === CALCULATE DERIVED METRICS ===
print("🔧 Calculating derived metrics...")

# 1. Cumulative GPA per student
cum_gpa = results.groupby('STUDENT ID', as_index=False)['GPA'].agg({
    'CUMULATIVE_GPA': 'mean',
    'TOTAL_SEMESTERS': 'count',
    'BEST_GPA': 'max',
    'WORST_GPA': 'min'
}).round(2)

# 2. Merge with profiles
df = profiles.merge(cum_gpa, on='STUDENT ID', how='left')

# 3. Calculate completion time and academic outcome
df['COMPLETION_MONTHS'] = (df['COMPLETION DATE'] - df['COMMENCEMENT DATE']).dt.days / 30.44
df['ACADEMIC_OUTCOME'] = np.where(df['CUMULATIVE_GPA'] >= 2.0, 'Pass', 'Fail')
df['PERFORMANCE_TIER'] = pd.cut(df['CUMULATIVE_GPA'], 
                               bins=[0, 2.0, 2.5, 3.0, 4.0], 
                               labels=['Poor', 'Average', 'Good', 'Excellent'])

# 4. NEW CALCULATED FIELDS - Advanced Analytics
print("🔧 Creating advanced calculated fields...")

# Academic Progress Metrics
df['GPA_IMPROVEMENT'] = df['BEST_GPA'] - df['WORST_GPA']  # Range of performance
df['GPA_VOLATILITY'] = df['GPA_IMPROVEMENT'] / df['TOTAL_SEMESTERS']  # Volatility per semester
df['ACADEMIC_EFFICIENCY'] = df['CUMULATIVE_GPA'] / df['COMPLETION_MONTHS']  # Performance per month
df['STUDY_INTENSITY'] = df['TOTAL_SEMESTERS'] / df['COMPLETION_MONTHS'] * 12  # Semesters per year

# Performance Categories
df['FAST_TRACK'] = np.where(df['COMPLETION_MONTHS'] <= 24, 'Fast Track', 
                   np.where(df['COMPLETION_MONTHS'] <= 36, 'Standard', 'Extended'))

# Financial & Demographic Scoring
funding_priority = {'Scholarship': 4, 'Sponsored': 3, 'Loan': 2, 'Self-Funded': 1}
df['FUNDING_PRIORITY'] = df['COURSE FUNDING'].map(funding_priority)

# Academic Risk Assessment
df['RISK_SCORE'] = (
    np.where(df['CUMULATIVE_GPA'] < 2.0, 3, 0) +  # Academic risk
    np.where(df['COMPLETION_MONTHS'] > 48, 2, 0) +  # Time risk
    np.where(df['GPA_VOLATILITY'] > 0.1, 1, 0)     # Consistency risk
)
df['RISK_CATEGORY'] = pd.cut(df['RISK_SCORE'], 
                            bins=[-1, 0, 2, 6], 
                            labels=['Low Risk', 'Medium Risk', 'High Risk'])

# Success Prediction Score (0-100)
df['SUCCESS_SCORE'] = (
    (df['CUMULATIVE_GPA'] / 4.0 * 40) +  # 40% weight on GPA
    (np.where(df['COMPLETION_MONTHS'] <= 36, 30, 
             np.where(df['COMPLETION_MONTHS'] <= 48, 20, 10))) +  # 30% on completion speed
    (df['FUNDING_PRIORITY'] * 5) +  # 20% on funding
    (np.where(df['GPA_VOLATILITY'] <= 0.05, 10, 
             np.where(df['GPA_VOLATILITY'] <= 0.1, 5, 0)))  # 10% on consistency
).round(1)

# 5. Prepare period-wise data for time series analysis
period_data = results.merge(profiles[['STUDENT ID', 'CITIZENSHIP', 'GENDER', 'COURSE FUNDING']], on='STUDENT ID')
period_summary = period_data.groupby(['PERIOD', 'CITIZENSHIP'], as_index=False)['GPA'].mean().round(2)

print(f"📈 Dataset Summary: {len(df)} students, {len(results)} semester records")
print("✅ All calculated fields completed!")
print("=" * 60)

🔧 Calculating derived metrics...
🔧 Creating advanced calculated fields...
📈 Dataset Summary: 285 students, 531 semester records
✅ All calculated fields completed!


# PART 2: INDIVIDUAL CHART CREATIONS
## Professional Analytics Charts with Template Aesthetics

This section creates 4 individual charts with modern styling, interactive elements, and comprehensive insights matching the dashboard template aesthetic.

In [4]:
# === CHART 1: PLOTLY EXPRESS - Student Distribution Treemap ===
print("🎨 Creating Chart 1: Student Distribution Treemap...")

# Prepare hierarchical data for treemap
treemap_data = df.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
treemap_data['PERCENTAGE'] = (treemap_data['COUNT'] / len(df) * 100).round(1)

fig1 = px.treemap(
    treemap_data,
    path=['CITIZENSHIP', 'GENDER', 'COURSE FUNDING'],
    values='COUNT',
    title='<b>🌍 Student Population Breakdown</b><br><sub>Distribution by Citizenship, Gender, and Funding Type • Click to drill down</sub>',
    color='COUNT',
    color_continuous_scale='Viridis',
    hover_data={'PERCENTAGE': ':.1f%'}
)

fig1.update_layout(
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    title_font_size=18,
    title_x=0.5,
    title_font_color=COLORS['light'],
    paper_bgcolor=COLORS['dark_bg'],
    height=700,
    margin=dict(t=150, l=20, r=20, b=20)
)

fig1.update_traces(
    textinfo='label+value+percent parent',
    textfont_size=11,
    textfont_color=COLORS['light'],
    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{customdata[0]:.1f}%<extra></extra>'
)

fig1.show()
print("✅ Chart 1 completed!")

🎨 Creating Chart 1: Student Distribution Treemap...


✅ Chart 1 completed!


In [5]:
# === CHART 2: INTERACTIVE GPA vs COMPLETION TIME ANALYSIS ===
print("🎨 Creating Chart 2: Interactive GPA vs Completion Time Analysis...")

# Prepare data for scatter plot analysis
scatter_data = df.dropna(subset=['CUMULATIVE_GPA', 'COMPLETION_MONTHS', 'SUCCESS_SCORE'])

# Create interactive scatter plot with multiple filtering options
fig2 = go.Figure()

# Create traces for each citizenship type (initially all visible)
citizenship_types = scatter_data['CITIZENSHIP'].unique()
for i, citizenship in enumerate(citizenship_types):
    citizenship_subset = scatter_data[scatter_data['CITIZENSHIP'] == citizenship]
    
    fig2.add_trace(go.Scatter(
        x=citizenship_subset['COMPLETION_MONTHS'],
        y=citizenship_subset['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (citizenship_subset['RISK_SCORE'] * 4),  # Size by risk score
            color=citizenship_subset['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=True if i == 0 else False,  # Show colorbar only once
            colorbar=dict(
                title="Success<br>Score<br>(0-100)",
                titlefont=dict(size=12, family='Arial Black'),
                tickfont=dict(size=10),
                len=0.8
            ),
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f}<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in citizenship_subset.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} ({len(citizenship_subset)} students)',
        visible=True
    ))

# Add HIGH ACHIEVERS traces (GPA > 3.5 AND completion < 5 months) for each citizenship
for i, citizenship in enumerate(citizenship_types):
    high_achievers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                 (scatter_data['CUMULATIVE_GPA'] > 3.5) &
                                 (scatter_data['COMPLETION_MONTHS'] < 5)]
    
    fig2.add_trace(go.Scatter(
        x=high_achievers['COMPLETION_MONTHS'],
        y=high_achievers['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (high_achievers['RISK_SCORE'] * 4),
            color=high_achievers['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=False,
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f} ⭐⭐<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months 🚀<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in high_achievers.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} High Achievers ({len(high_achievers)} students)',
        visible=False
    ))

# Add UNDERPERFORMERS traces (GPA < 2.5) for each citizenship
for i, citizenship in enumerate(citizenship_types):
    underperformers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                  (scatter_data['CUMULATIVE_GPA'] < 2.5)]
    
    fig2.add_trace(go.Scatter(
        x=underperformers['COMPLETION_MONTHS'],
        y=underperformers['CUMULATIVE_GPA'],
        mode='markers',
        marker=dict(
            size=12 + (underperformers['RISK_SCORE'] * 4),
            color=underperformers['SUCCESS_SCORE'],
            colorscale='Viridis',
            showscale=False,
            line=dict(width=2, color='white'),
            opacity=0.8,
            symbol='circle'
        ),
        text=[f"Student ID: {row['STUDENT ID']}<br>" +
              f"GPA: {row['CUMULATIVE_GPA']:.2f} 🚨<br>" +
              f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
              f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
              f"Risk Category: {row['RISK_CATEGORY']}<br>" +
              f"Citizenship: {row['CITIZENSHIP']}<br>" +
              f"Gender: {row['GENDER']}<br>" +
              f"Funding: {row['COURSE FUNDING']}"
              for _, row in underperformers.iterrows()],
        hovertemplate='%{text}<extra></extra>',
        name=f'{citizenship} Underperformers ({len(underperformers)} students)',
        visible=False
    ))

# Create dropdown options for different views
num_citizenship = len(citizenship_types)
dropdown_buttons = [
    # Show all citizenship types (regular view)
    dict(label='🌍 All Citizenship Types', 
         method='update', 
         args=[{'visible': [True] * num_citizenship + [False] * (num_citizenship * 2)}]),
]

# Add individual citizenship options (regular view)
for i, citizenship in enumerate(citizenship_types):
    visibility = [False] * (num_citizenship * 3)  # Total traces = citizenship + high achievers + underperformers
    visibility[i] = True  # Show only this citizenship's regular trace
    dropdown_buttons.append(
        dict(label=f'🎯 {citizenship} Only',
             method='update',
             args=[{'visible': visibility}])
    )

# Add high achievers view (show all high achiever traces)
high_achiever_visibility = [False] * num_citizenship + [True] * num_citizenship + [False] * num_citizenship
dropdown_buttons.append(
    dict(label='⭐ High Achievers (GPA > 3.5 & < 5 months)', 
         method='update', 
         args=[{'visible': high_achiever_visibility}])
)

# Add underperformers view (show all underperformer traces)
underperformer_visibility = [False] * (num_citizenship * 2) + [True] * num_citizenship
dropdown_buttons.append(
    dict(label='🚨 Underperformers (GPA < 2.5)', 
         method='update', 
         args=[{'visible': underperformer_visibility}])
)

# Configure layout with enhanced interactivity and proper filter positioning
fig2.update_layout(
    title=dict(
        text='<b>🎯 Interactive GPA vs Completion Time Analysis</b><br>' +
             '<sub>Success score by color • Risk level by marker size • Use filters to explore different groups</sub>',
        font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
        x=0.5,
        y=0.95
    ),
    xaxis=dict(
        title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
        tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[0, max(scatter_data['COMPLETION_MONTHS']) + 5]
    ),
    yaxis=dict(
        title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
        tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[1.0, 4.0]
    ),
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    paper_bgcolor=COLORS['dark_bg'],
    plot_bgcolor='rgba(0,0,0,0)',
    height=700,
    margin=dict(t=150, l=80, r=120, b=80),
    showlegend=True,
    legend=dict(
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='left',
        x=1.02,
        bgcolor=COLORS['card_bg'],
        bordercolor=COLORS['neon_blue'],
        borderwidth=1,
        font=dict(color=COLORS['light'])
    ),
    
    # Add interactive dropdown menu with proper positioning
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
            x=0.02,
            xanchor="left",
            y=1.12,
            yanchor="top",
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_green'],
            borderwidth=2,
            font=dict(color=COLORS['light'])
        )
    ],
    
    # Add filter label with proper positioning
    annotations=[
        dict(
            text="🔍 Filter by Group:",
            x=0.02, y=1.18,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
        )
    ]
)

# Add reference lines with dashboard colors
fig2.add_hline(y=2.0, line_dash="dot", line_color=COLORS['danger'], line_width=2, opacity=0.8, 
              annotation_text="Pass Threshold (2.0)")
fig2.add_hline(y=3.5, line_dash="dot", line_color=COLORS['success'], line_width=2, opacity=0.8, 
              annotation_text="Excellence Threshold (3.5)")

# Add quadrant background shading with dashboard colors
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=3.5, x1=1, y1=4.0,
               fillcolor=f"rgba({int(COLORS['success'][1:3], 16)}, {int(COLORS['success'][3:5], 16)}, {int(COLORS['success'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=2.0, x1=1, y1=3.5,
               fillcolor=f"rgba({int(COLORS['secondary'][1:3], 16)}, {int(COLORS['secondary'][3:5], 16)}, {int(COLORS['secondary'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)
fig2.add_shape(type="rect", xref="paper", yref="y", x0=0, y0=1.0, x1=1, y1=2.0,
               fillcolor=f"rgba({int(COLORS['danger'][1:3], 16)}, {int(COLORS['danger'][3:5], 16)}, {int(COLORS['danger'][5:7], 16)}, 0.1)", 
               layer="below", line_width=0)

fig2.show()
print("✅ Chart 2 completed!")

🎨 Creating Chart 2: Interactive GPA vs Completion Time Analysis...


✅ Chart 2 completed!


In [6]:
# === CHART 3: INTERACTIVE BOX PLOT - COMPLETION TIME ANALYSIS ===
print("🎨 Creating Chart 3: Interactive Completion Time Box Plot...")

# Filter out invalid completion times
completion_data = df.dropna(subset=['COMPLETION_MONTHS'])
completion_data = completion_data[completion_data['COMPLETION_MONTHS'] > 0]
completion_data = completion_data[completion_data['COMPLETION_MONTHS'] <= 60]  # Reasonable max of 5 years

fig3 = go.Figure()

# Add box plot for all citizenship types
for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
    citizenship_data = completion_data[completion_data['CITIZENSHIP'] == citizenship]
    
    fig3.add_trace(
        go.Box(
            y=citizenship_data['COMPLETION_MONTHS'],
            name=citizenship,
            marker_color=citizenship_colors[i % len(citizenship_colors)],
            boxpoints='outliers',  # Show outliers
            jitter=0.3,  # Add some jitter to points
            pointpos=-1.8,  # Position of points
            marker=dict(
                size=4,
                opacity=0.6,
                line=dict(width=1, color='white')
            ),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Completion Time: %{y:.1f} months<br>' +
                         'Student Count: ' + str(len(citizenship_data)) + '<br>' +
                         '<extra></extra>',
            visible=True
        )
    )

# Add "All Categories" option (show all)
all_visible = [True] * len(completion_data['CITIZENSHIP'].unique())

# Create dropdown options
dropdown_buttons = [
    dict(label='All Citizenship Types',
         method='update',
         args=[{'visible': all_visible}])
]

# Add individual citizenship type options
for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
    visibility = [False] * len(completion_data['CITIZENSHIP'].unique())
    visibility[i] = True
    dropdown_buttons.append(
        dict(label=f'{citizenship} Only',
             method='update',
             args=[{'visible': visibility}])
    )

fig3.update_layout(
    title='<b>🔍 Course Completion Time Distribution</b><br><sub>Interactive box plot analysis of completion duration by citizenship status</sub>',
    title_font_size=18,
    title_x=0.5,
    title_font_color=COLORS['light'],
    yaxis_title='Completion Time (Months)',
    xaxis_title='Citizenship Status',
    font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
    paper_bgcolor=COLORS['dark_bg'],
    plot_bgcolor='rgba(0,0,0,0)',
    height=700,
    margin=dict(t=150, l=80, r=80, b=80),
    showlegend=False,
    updatemenus=[
        dict(
            buttons=dropdown_buttons,
            direction='down',
            showactive=True,
            x=0.02,
            xanchor='left',
            y=1.12,
            yanchor='top',
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_purple'],
            borderwidth=2,
            font=dict(color=COLORS['light'])
        )
    ],
    annotations=[
        dict(
            text="🔍 Filter Options:",
            x=0.02, y=1.18,
            xref="paper", yref="paper",
            showarrow=False,
            font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_purple'])
        )
    ]
)

# Add grid lines for better readability with dashboard theme
fig3.update_layout(
    yaxis=dict(
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        range=[0, 60],
        title_font_color=COLORS['neon_green'],
        tickfont=dict(color=COLORS['text'])
    ),
    xaxis=dict(
        gridcolor='rgba(200,200,200,0.2)',
        gridwidth=1,
        title_font_color=COLORS['neon_blue'],
        tickfont=dict(color=COLORS['text'])
    )
)

fig3.show()
print("✅ Chart 3 completed!")

🎨 Creating Chart 3: Interactive Completion Time Box Plot...


✅ Chart 3 completed!


In [7]:
# === CHART 4: ANIMATED GPA TRENDS OVER TIME ===
print("🎨 Creating Chart 4: Animated GPA Performance Trends...")

# Use the existing period_data which has semester GPA information
gpa_trends = period_data.groupby(['PERIOD', 'CITIZENSHIP'], as_index=False).agg({
    'GPA': ['mean', 'count']
}).round(2)

# Flatten column names
gpa_trends.columns = ['PERIOD', 'CITIZENSHIP', 'AVG_GPA', 'STUDENT_COUNT']

# Filter out periods/citizenship combinations with very few students (less than 3)
gpa_trends = gpa_trends[gpa_trends['STUDENT_COUNT'] >= 3]

# Get unique periods and citizenship types for consistent animation
periods = sorted(gpa_trends['PERIOD'].unique())
citizenship_types = sorted(gpa_trends['CITIZENSHIP'].unique())

# Create frames for animation with enhanced visuals
frames = []
for period in periods:
    period_frame = gpa_trends[gpa_trends['PERIOD'] == period]
    
    if len(period_frame) > 0:  # Only create frame if data exists
        frames.append(
            go.Frame(
                data=[
                    go.Scatter(
                        x=period_frame['CITIZENSHIP'],
                        y=period_frame['AVG_GPA'],
                        mode='markers+lines+text',
                        marker=dict(
                            size=[20 + (count/5) for count in period_frame['STUDENT_COUNT']],  # Size based on student count
                            color=citizenship_colors[:len(period_frame)],
                            line=dict(width=3, color='white'),
                            opacity=0.8,
                            symbol='circle'
                        ),
                        line=dict(
                            width=4, 
                            color=COLORS['primary'],
                            shape='spline',  # Smooth curved lines
                            smoothing=0.3
                        ),
                        text=[f"{gpa:.2f}" for gpa in period_frame['AVG_GPA']],
                        textposition='top center',
                        textfont=dict(size=14, color=COLORS['text'], family='Arial Black'),
                        hovertemplate='<b>%{x}</b><br>' +
                                    'Average GPA: %{y:.2f}<br>' +
                                    'Students: %{customdata}<br>' +
                                    'Period: ' + period + '<extra></extra>',
                        customdata=period_frame['STUDENT_COUNT'],
                        name=period,
                        connectgaps=True
                    )
                ],
                name=period
            )
        )

# Create initial frame with better styling
if len(periods) > 0:
    initial_period_data = gpa_trends[gpa_trends['PERIOD'] == periods[0]]
    
    fig4 = go.Figure(
        data=[
            go.Scatter(
                x=initial_period_data['CITIZENSHIP'],
                y=initial_period_data['AVG_GPA'],
                mode='markers+lines+text',
                marker=dict(
                    size=[20 + (count/5) for count in initial_period_data['STUDENT_COUNT']],
                    color=citizenship_colors[:len(initial_period_data)],
                    line=dict(width=3, color='white'),
                    opacity=0.8,
                    symbol='circle'
                ),
                line=dict(
                    width=4, 
                    color=COLORS['primary'],
                    shape='spline',
                    smoothing=0.3
                ),
                text=[f"{gpa:.2f}" for gpa in initial_period_data['AVG_GPA']],
                textposition='top center',
                textfont=dict(size=14, color=COLORS['text'], family='Arial Black'),
                hovertemplate='<b>%{x}</b><br>' +
                            'Average GPA: %{y:.2f}<br>' +
                            'Students: %{customdata}<br>' +
                            'Period: ' + periods[0] + '<extra></extra>',
                customdata=initial_period_data['STUDENT_COUNT'],
                name=periods[0],
                connectgaps=True
            )
        ],
        frames=frames
    )

    # Enhanced layout with dashboard theme
    fig4.update_layout(
        title=dict(
            text='<b>📈 Academic Performance Evolution Over Time</b><br>' +
                 '<sub>Average GPA trends by citizenship status across semesters</sub><br>' +
                 '<span style="font-size:12px; color:' + COLORS['text_dim'] + ';">💡 Marker size reflects student count | Only periods with 3+ students shown</span>',
            font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
            x=0.5,
            y=0.95
        ),
        xaxis=dict(
            title=dict(text='Citizenship Status', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            showline=True,
            linewidth=2,
            linecolor=COLORS['text_dim'],
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text'])
        ),
        yaxis=dict(
            title=dict(text='Average GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
            range=[1.5, 4.0], 
            gridcolor='rgba(200,200,200,0.2)', 
            gridwidth=1,
            tickmode='linear',
            tick0=1.5,
            dtick=0.25,
            showline=True,
            linewidth=2,
            linecolor=COLORS['text_dim'],
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text'])
        ),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=90, r=90, b=80),
        showlegend=False,
        
        # Enhanced animation controls with dashboard theme
        updatemenus=[
            dict(
                type='buttons',
                showactive=False,
                x=0.02,
                xanchor='left',
                y=1.12,
                yanchor='top',
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_green'],
                borderwidth=2,
                font=dict(color=COLORS['light']),
                buttons=[
                    dict(
                        label='▶️ Play Animation',
                        method='animate',
                        args=[None, dict(
                            frame=dict(duration=1200, redraw=True), 
                            fromcurrent=True,
                            transition=dict(duration=400, easing='cubic-in-out')
                        )]
                    ),
                    dict(
                        label='⏸️ Pause',
                        method='animate',
                        args=[[None], dict(
                            frame=dict(duration=0, redraw=False), 
                            mode='immediate'
                        )]
                    ),
                    dict(
                        label='🔄 Restart',
                        method='animate',
                        args=[None, dict(
                            frame=dict(duration=1200, redraw=True), 
                            fromcurrent=False,
                            transition=dict(duration=400, easing='cubic-in-out')
                        )]
                    )
                ]
            )
        ],
        
        # Add filter label annotation
        annotations=[
            dict(
                text="🎬 Animation Controls:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
            )
        ],
        
        # Enhanced slider with better styling
        sliders=[
            dict(
                active=0,
                yanchor='top',
                xanchor='left',
                currentvalue=dict(
                    font=dict(size=16, family='Arial Black'), 
                    prefix='📅 Current Period: ', 
                    visible=True, 
                    xanchor='center'
                ),
                transition=dict(duration=400, easing='cubic-in-out'),
                pad=dict(b=10, t=60),
                len=0.85,
                x=0.075,
                y=-0.05,
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor=COLORS['neutral'],
                borderwidth=1,
                tickcolor=COLORS['primary'],
                steps=[
                    dict(
                        args=[[period], dict(
                            frame=dict(duration=400, redraw=True), 
                            mode='immediate',
                            transition=dict(duration=400, easing='cubic-in-out')
                        )],
                        label=period,
                        method='animate',
                        value=period
                    ) for period in periods
                ]
            )
        ]
    )

    # Add enhanced reference lines with dashboard colors
    fig4.add_hline(
        y=2.0, 
        line_dash="dot", 
        line_color=COLORS['danger'], 
        line_width=3,
        annotation_text="🎯 Pass Threshold (2.0)",
        annotation_position="top right"
    )

    fig4.add_hline(
        y=3.0, 
        line_dash="dot", 
        line_color=COLORS['success'], 
        line_width=3,
        annotation_text="🌟 Excellence Threshold (3.0)",
        annotation_position="bottom right"
    )

    # Add subtle gradient effect with shapes using dashboard colors
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=3.0, x1=1, y1=4.0,
        fillcolor=f"rgba({int(COLORS['success'][1:3], 16)}, {int(COLORS['success'][3:5], 16)}, {int(COLORS['success'][5:7], 16)}, 0.1)",
        layer="below",
        line_width=0,
    )
    
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=2.0, x1=1, y1=3.0,
        fillcolor="rgba(52, 152, 219, 0.1)",
        layer="below",
        line_width=0,
    )
    
    fig4.add_shape(
        type="rect",
        xref="paper", yref="y",
        x0=0, y0=1.5, x1=1, y1=2.0,
        fillcolor="rgba(231, 76, 60, 0.1)",
        layer="below",
        line_width=0,
    )

    fig4.show()
    print("✅ Chart 4 completed!")
else:
    print("⚠️ No sufficient data for GPA trends animation")

🎨 Creating Chart 4: Animated GPA Performance Trends...


✅ Chart 4 completed!


# PART 3: ANALYTICS INSIGHTS & DASHBOARD INTEGRATION
## Summary Statistics and Actionable Insights

This section provides comprehensive insights from the data analysis and prepares the foundation for dashboard integration.

In [8]:
# === COMPREHENSIVE ANALYTICS INSIGHTS ===
print("\n" + "="*60)
print("📊 COMPREHENSIVE ANALYTICS INSIGHTS")
print("="*60)

# Analyze and print insights focusing on GPA vs Completion Time
print("\n🎯 GPA vs COMPLETION TIME INSIGHTS:")
print("=" * 50)

print("📊 SUCCESS SCORE DISTRIBUTION:")
success_quartiles = scatter_data['SUCCESS_SCORE'].quantile([0.25, 0.5, 0.75])
print(f"   • Top 25%: Success Score ≥ {success_quartiles[0.75]:.1f}")
print(f"   • Median: {success_quartiles[0.5]:.1f}")
print(f"   • Bottom 25%: Success Score ≤ {success_quartiles[0.25]:.1f}")

print(f"\n🚨 RISK ASSESSMENT BREAKDOWN:")
risk_breakdown = scatter_data['RISK_CATEGORY'].value_counts()
for risk, count in risk_breakdown.items():
    percentage = (count / len(scatter_data)) * 100
    avg_completion = scatter_data[scatter_data['RISK_CATEGORY'] == risk]['COMPLETION_MONTHS'].mean()
    avg_gpa = scatter_data[scatter_data['RISK_CATEGORY'] == risk]['CUMULATIVE_GPA'].mean()
    print(f"   • {risk}: {count} students ({percentage:.1f}%) - Avg: {avg_gpa:.2f} GPA, {avg_completion:.1f} months")

print(f"\n⏰ COMPLETION TIME vs GPA ANALYSIS:")
fast_students = scatter_data[scatter_data['COMPLETION_MONTHS'] <= 30]
slow_students = scatter_data[scatter_data['COMPLETION_MONTHS'] > 48]
print(f"   • Fast completion (≤30 months): {len(fast_students)} students, Avg GPA: {fast_students['CUMULATIVE_GPA'].mean():.2f}")
print(f"   • Slow completion (>48 months): {len(slow_students)} students, Avg GPA: {slow_students['CUMULATIVE_GPA'].mean():.2f}")

print(f"\n🎯 HIGH PERFORMERS vs LOW PERFORMERS:")
high_gpa = scatter_data[(scatter_data['CUMULATIVE_GPA'] > 3.5) & (scatter_data['COMPLETION_MONTHS'] < 5)]
low_gpa = scatter_data[scatter_data['CUMULATIVE_GPA'] < 2.0]
print(f"   • Elite High Achievers (GPA > 3.5 & < 5 months): {len(high_gpa)} students")
print(f"   • Low GPA (<2.0): {len(low_gpa)} students, Avg completion: {low_gpa['COMPLETION_MONTHS'].mean():.1f} months")

print(f"\n🎯 KEY RECOMMENDATIONS:")
high_risk_students = scatter_data[scatter_data['RISK_CATEGORY'] == 'High Risk']
if len(high_risk_students) > 0:
    print(f"   • {len(high_risk_students)} students need immediate academic support")
    
fast_excellent = scatter_data[(scatter_data['COMPLETION_MONTHS'] < 5) & (scatter_data['CUMULATIVE_GPA'] > 3.5)]
if len(fast_excellent) > 0:
    print(f"   • {len(fast_excellent)} students are elite high achievers (GPA > 3.5 + completion < 5 months)")
    
at_risk = scatter_data[(scatter_data['CUMULATIVE_GPA'] < 2.5) & (scatter_data['COMPLETION_MONTHS'] > 42)]
print(f"   • {len(at_risk)} students are struggling (low GPA + slow completion)")

print("✅ Detailed insights analysis completed!")


📊 COMPREHENSIVE ANALYTICS INSIGHTS

🎯 GPA vs COMPLETION TIME INSIGHTS:
📊 SUCCESS SCORE DISTRIBUTION:
   • Top 25%: Success Score ≥ 89.2
   • Median: 84.0
   • Bottom 25%: Success Score ≤ 80.0

🚨 RISK ASSESSMENT BREAKDOWN:
   • Low Risk: 40 students (71.4%) - Avg: 3.25 GPA, 5.9 months
   • Medium Risk: 16 students (28.6%) - Avg: 2.94 GPA, 17.0 months
   • High Risk: 0 students (0.0%) - Avg: nan GPA, nan months

⏰ COMPLETION TIME vs GPA ANALYSIS:
   • Fast completion (≤30 months): 56 students, Avg GPA: 3.16
   • Slow completion (>48 months): 0 students, Avg GPA: nan

🎯 HIGH PERFORMERS vs LOW PERFORMERS:
   • Elite High Achievers (GPA > 3.5 & < 5 months): 5 students
   • Low GPA (<2.0): 0 students, Avg completion: nan months

🎯 KEY RECOMMENDATIONS:
   • 5 students are elite high achievers (GPA > 3.5 + completion < 5 months)
   • 0 students are struggling (low GPA + slow completion)
✅ Detailed insights analysis completed!


In [9]:
# === SUMMARY STATISTICS FOR DASHBOARD INTEGRATION ===
print("\n" + "="*60)
print("📊 KEY METRICS SUMMARY FOR DASHBOARD")
print("="*60)

# Calculate key metrics
total_students = len(df)
avg_completion_time = df['COMPLETION_MONTHS'].mean()
avg_gpa = df['CUMULATIVE_GPA'].mean()
pass_rate = (df['ACADEMIC_OUTCOME'] == 'Pass').mean() * 100
best_citizenship = df.groupby('CITIZENSHIP')['CUMULATIVE_GPA'].mean().idxmax()
best_funding = df.groupby('COURSE FUNDING')['CUMULATIVE_GPA'].mean().idxmax()
fastest_completion = df.groupby('CITIZENSHIP')['COMPLETION_MONTHS'].mean().idxmin()

print(f"📈 Total Students Analyzed: {total_students:,}")
print(f"⏱️ Average Completion Time: {avg_completion_time:.1f} months")
print(f"🎯 Overall Average GPA: {avg_gpa:.2f}")
print(f"✅ Pass Rate (GPA ≥ 2.0): {pass_rate:.1f}%")
print(f"🏆 Top Performing Citizenship: {best_citizenship}")
print(f"💰 Best Funding Type: {best_funding}")
print(f"⚡ Fastest Completion Group: {fastest_completion}")
print(f"📚 Analysis covers enrollment and completion patterns")

print("\n🎨 VISUALIZATION SYSTEM COMPLETED!")
print("Each chart features:")
print("• Chart 1: Population distribution treemap")
print("• Chart 2: Interactive GPA vs completion time correlation analysis")
print("• Chart 3: Course completion time distribution patterns")
print("• Chart 4: Animated GPA trends over semesters")
print("• Interactive elements (dropdowns, sliders, play/pause)")
print("• Comprehensive hover information")
print("• Actionable insights for decision-making")

print(f"\n🔗 DATA PREPARATION STATUS:")
print("• ✅ All datasets loaded and processed")
print("• ✅ Advanced calculated fields generated")
print("• ✅ Risk assessment scoring completed")
print("• ✅ Success prediction metrics calculated")
print("• ✅ Interactive visualizations created")
print("• ✅ Template aesthetic consistency maintained")
print("• ✅ Ready for dashboard integration")

print(f"\n📊 DASHBOARD INTEGRATION READY!")
print("This analysis system can now be integrated with:")
print("• Dash web application framework")
print("• Real-time filtering capabilities")
print("• Student search functionality")
print("• KPI monitoring dashboards")
print("• Administrative decision support tools")


📊 KEY METRICS SUMMARY FOR DASHBOARD
📈 Total Students Analyzed: 285
⏱️ Average Completion Time: 9.5 months
🎯 Overall Average GPA: 3.16
✅ Pass Rate (GPA ≥ 2.0): 95.4%
🏆 Top Performing Citizenship: FOREIGNER
💰 Best Funding Type: Individual - SFC + $1000 SCHOLARSHIP
⚡ Fastest Completion Group: SG PR
📚 Analysis covers enrollment and completion patterns

🎨 VISUALIZATION SYSTEM COMPLETED!
Each chart features:
• Chart 1: Population distribution treemap
• Chart 2: Interactive GPA vs completion time correlation analysis
• Chart 3: Course completion time distribution patterns
• Chart 4: Animated GPA trends over semesters
• Interactive elements (dropdowns, sliders, play/pause)
• Comprehensive hover information
• Actionable insights for decision-making

🔗 DATA PREPARATION STATUS:
• ✅ All datasets loaded and processed
• ✅ Advanced calculated fields generated
• ✅ Risk assessment scoring completed
• ✅ Success prediction metrics calculated
• ✅ Interactive visualizations created
• ✅ Template aesthetic 

# PART 4: INTERACTIVE DASHBOARD IMPLEMENTATION
## Futuristic Analytics Dashboard with Dash Framework

This section integrates all the analysis and charts into a live, interactive web dashboard with search functionality and real-time KPI monitoring.

In [10]:
# === DASHBOARD APP INITIALIZATION (MATCHING TEMPLATE) ===
print("🚀 Initializing Futuristic Analytics Dashboard...")

# Import Dash framework
from dash import Dash, html, dcc, Input, Output, State, callback
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

# ================================
# APP INITIALIZATION WITH DATE PICKER FIX
# ================================
app = Dash(
    __name__,
    external_stylesheets=[
        dbc.themes.CYBORG,  # Futuristic dark theme
        'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css'
    ],
    suppress_callback_exceptions=True
)

app.title = "Student Analytics Dashboard"

# Custom CSS to fix date picker z-index overlay issues
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .DateInput_input, .DateRangePickerInput {
                z-index: 9999 !important;
                position: relative !important;
            }
            .DateRangePicker, .DayPicker {
                z-index: 10000 !important;
                position: absolute !important;
            }
            .SingleDatePicker_picker {
                z-index: 10004 !important;
                position: absolute !important;
            }
            .SingleDatePickerInput {
                z-index: 10006 !important;
                position: relative !important;
            }
            .DayPicker_portal {
                z-index: 10002 !important;
            }
            .DateInput_fang {
                z-index: 10003 !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

print("✅ Dashboard framework initialized with date picker overlay fix!")

🚀 Initializing Futuristic Analytics Dashboard...
✅ Dashboard framework initialized with date picker overlay fix!


In [11]:
# === DASHBOARD HEADER COMPONENT (MATCHING TEMPLATE) ===
print("🎨 Creating futuristic header component...")

# Header with enhanced glowing effects - EXACT TEMPLATE MATCH
header = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.Div([
                    html.I(className="fas fa-graduation-cap", 
                          style={
                              'fontSize': '1.8rem', 
                              'color': COLORS['neon_blue'],
                              'filter': 'drop-shadow(0 0 8px rgba(0, 245, 255, 0.8))'
                          }),
                ], style={
                    'width': '50px', 'height': '50px', 'borderRadius': '15px',
                    'background': f'linear-gradient(135deg, {COLORS["primary"]} 0%, {COLORS["neon_blue"]} 100%)',
                    'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center',
                    'marginRight': '15px', 
                    'boxShadow': f'0 0 25px {COLORS["glow_shadow"]}, 0 0 50px rgba(139, 95, 191, 0.3)',
                    'border': f'1px solid {COLORS["neon_blue"]}',
                    'animation': 'pulse 2s infinite'
                }),
                html.Div([
                    html.H5("STUDENT ANALYTICS", style={
                        'color': COLORS['light'], 'margin': '0', 'fontWeight': '700',
                        'letterSpacing': '1px', 'fontSize': '0.95rem',
                        'textShadow': f'0 0 10px {COLORS["neon_blue"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    html.P("CONTROL CENTER", style={
                        'color': COLORS['neon_blue'], 'margin': '0', 'fontSize': '0.75rem',
                        'fontWeight': '500', 'letterSpacing': '0.5px',
                        'fontFamily': 'Rajdhani, monospace'
                    })
                ])
            ], style={'display': 'flex', 'alignItems': 'center'})
        ], width=3),
        
        dbc.Col([
            html.H1("ANALYTICS NEXUS", 
                   className="text-center",
                   style={
                       'color': COLORS['light'], 
                       'fontWeight': '900',
                       'fontSize': '2.5rem',
                       'margin': '0',
                       'background': f'linear-gradient(135deg, {COLORS["neon_blue"]} 0%, {COLORS["neon_purple"]} 50%, {COLORS["primary"]} 100%)',
                       'backgroundClip': 'text',
                       'WebkitBackgroundClip': 'text',
                       'WebkitTextFillColor': 'transparent',
                       'letterSpacing': '2px',
                       'textShadow': '0 0 20px rgba(34, 211, 238, 0.6)',
                       'filter': 'drop-shadow(0 0 10px rgba(34, 211, 238, 0.8))',
                       'fontFamily': 'Orbitron, monospace'
                   })
        ], width=6),
        
        dbc.Col([
            html.Div([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-circle", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.6rem',
                            'animation': 'blink 1.5s infinite'
                        }),
                        html.Span("LIVE", style={
                            'color': COLORS['neon_green'], 'fontSize': '0.8rem', 
                            'marginLeft': '6px', 'fontWeight': '700',
                            'textShadow': f'0 0 8px {COLORS["neon_green"]}',
                            'fontFamily': 'Orbitron, monospace'
                        })
                    ], style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '4px'}),
                    html.Span(id="live-time", style={
                        'color': COLORS['text'], 'fontSize': '0.85rem',
                        'fontFamily': 'Rajdhani, monospace', 'fontWeight': '500'
                    })
                ], style={'textAlign': 'right'})
            ])
        ], width=3)
    ], className="align-items-center"),
], fluid=True, className="py-4", 
   style={
       'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)', 
       'marginBottom': '30px',
       'borderRadius': '0 0 30px 30px',
       'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 0 50px rgba(26, 26, 46, 0.8)',
       'border': f'1px solid {COLORS["neon_blue"]}20',
       'position': 'relative',
       'overflow': 'hidden'
   })

print("✅ Header component created!")

🎨 Creating futuristic header component...
✅ Header component created!


In [12]:
# === MAIN LAYOUT (EXACT TEMPLATE MATCH) ===
print("🎨 Creating main layout to match template...")

# Chart containers with enhanced glowing design (matching template)
chart_style = {
    'background': f'linear-gradient(135deg, {COLORS["card_bg"]} 0%, {COLORS["sidebar_bg"]} 100%)',
    'border': f'2px solid {COLORS["neon_blue"]}40',
    'borderRadius': '25px',
    'padding': '30px',
    'marginBottom': '30px',
    'boxShadow': f'0 0 40px {COLORS["glow_shadow"]}, 0 15px 60px rgba(26, 26, 46, 0.8)',
    'backdropFilter': 'blur(15px)',
    'position': 'relative',
    'zIndex': '1',
    'overflow': 'hidden',
    'transition': 'all 0.3s ease'
}

# ================================
# MAIN LAYOUT WITH LEFT-RIGHT ORIENTATION (EXACT TEMPLATE MATCH)
# ================================
app.layout = html.Div([
    # Global CSS for futuristic fonts and date picker fix
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Rajdhani:wght@300;400;500;600;700&family=Space+Mono:wght@400;700&display=swap'
    ),
    
    # Live update interval
    dcc.Interval(id='interval-update', interval=60000, n_intervals=0),
    
    # Data store
    dcc.Store(id='filtered-data'),
    
    # Header (full width)
    header,
    
    # Main Content Container with Left-Right Layout
    dbc.Container([
        dbc.Row([
            # LEFT SIDE - Search, Student Data, and KPIs (4/12 columns)
            dbc.Col([
                # Search Section
                html.Div([
                    html.Label("🔍 SEARCH STUDENT ID", style={
                        'color': COLORS['pink'], 'fontWeight': '700', 'marginBottom': '15px',
                        'fontSize': '1rem', 'letterSpacing': '1px',
                        'textShadow': f'0 0 8px {COLORS["pink"]}',
                        'fontFamily': 'Orbitron, monospace'
                    }),
                    dbc.Input(
                        id='search-box',
                        placeholder="Enter Student ID (e.g., STU00001)",
                        type="text",
                        style={
                            'backgroundColor': COLORS['sidebar_bg'], 'color': COLORS['light'], 
                            'border': f'2px solid {COLORS["pink"]}60', 'borderRadius': '10px',
                            'boxShadow': f'inset 0 0 10px rgba(236, 72, 153, 0.2)',
                            'fontFamily': 'Rajdhani, monospace',
                            'fontSize': '1rem'
                        }
                    )
                ], style={
                    'backgroundColor': COLORS['card_bg'], 'padding': '20px', 'borderRadius': '20px',
                    'boxShadow': f'0 0 30px rgba(236, 72, 153, 0.4), 0 8px 32px rgba(26, 26, 46, 0.6)', 
                    'border': f'2px solid {COLORS["pink"]}40',
                    'backdropFilter': 'blur(10px)',
                    'marginBottom': '20px'
                }),
                
                # Student Data Section
                html.Div([
                    html.H4("📊 STUDENT DATA", 
                           style={
                               'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='student-data-table')
                ], style={**chart_style, 'height': '300px', 'overflow': 'auto', 'marginBottom': '20px'}),
                
                # KPI Cards Section (Compact vertical layout)
                html.Div([
                    html.H4("📈 KEY METRICS", 
                           style={
                               'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                               'fontWeight': '900', 'fontSize': '1.1rem', 'letterSpacing': '2px',
                               'fontFamily': 'Orbitron, monospace',
                               'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                               'textTransform': 'uppercase'
                           }),
                    html.Div(id='kpi-cards-left')
                ], style=chart_style)
                
            ], width=4),  # Left side takes 4/12 columns
            
            # RIGHT SIDE - Scrollable Charts (8/12 columns)
            dbc.Col([
                # Simple Date Filter Row with Fixed Z-Index
                html.Div([
                    html.Span("📅 Filter: ", style={
                        'color': COLORS['secondary'], 'fontWeight': '700', 'marginRight': '10px',
                        'fontFamily': 'Orbitron, monospace', 'fontSize': '0.9rem'
                    }),
                    html.Div([
                        dcc.DatePickerSingle(
                            id='start-date-picker',
                            date=datetime(2022, 4, 1),  # Start of actual data range
                            display_format='YYYY-MM-DD',
                            style={
                                'display': 'inline-block', 
                                'marginRight': '10px',
                                'zIndex': '9999'  # High z-index for calendar dropdown
                            }
                        )
                    ], style={
                        'display': 'inline-block', 
                        'position': 'relative',
                        'zIndex': '9999'  # Container z-index
                    }),
                    html.Span(" to ", style={'color': COLORS['light'], 'margin': '0 5px'}),
                    html.Div([
                        dcc.DatePickerSingle(
                            id='end-date-picker',
                            date=datetime(2025, 4, 24),  # End of actual data range
                            display_format='YYYY-MM-DD',
                            style={
                                'display': 'inline-block', 
                                'marginRight': '15px',
                                'zIndex': '9999'  # High z-index for calendar dropdown
                            }
                        )
                    ], style={
                        'display': 'inline-block', 
                        'position': 'relative',
                        'zIndex': '9999'  # Container z-index
                    }),
                    dbc.Button("All Time", id="btn-all-years", color="outline-info", size="sm",
                              style={'fontFamily': 'Rajdhani, monospace', 'fontSize': '0.8rem'})
                ], style={
                    'backgroundColor': COLORS['card_bg'], 
                    'padding': '10px 15px', 
                    'borderRadius': '10px',
                    'border': f'1px solid {COLORS["secondary"]}40', 
                    'marginBottom': '15px',
                    'display': 'flex', 
                    'alignItems': 'center', 
                    'flexWrap': 'wrap',
                    'position': 'relative',
                    'zIndex': '1000'  # Base z-index for filter container
                }),
                
                # Scrollable Charts Container
                html.Div([
                    # Chart 1 - Your Treemap
                    html.Div([
                        html.H4("POPULATION DISTRIBUTION", 
                               style={
                                   'color': COLORS['neon_blue'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_blue"]}, 0 0 30px {COLORS["neon_blue"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-1', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 2 - Your Scatter
                    html.Div([
                        html.H4("GPA vs COMPLETION ANALYSIS", 
                               style={
                                   'color': COLORS['neon_purple'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_purple"]}, 0 0 30px {COLORS["neon_purple"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-2', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 3 - Your Boxplot
                    html.Div([
                        html.H4("COMPLETION TIME ANALYSIS", 
                               style={
                                   'color': COLORS['neon_green'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["neon_green"]}, 0 0 30px {COLORS["neon_green"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-3', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style={**chart_style, 'marginBottom': '40px'}),
                    
                    # Chart 4 - Your Animation
                    html.Div([
                        html.H4("TEMPORAL TRENDS", 
                               style={
                                   'color': COLORS['accent'], 'textAlign': 'center', 'marginBottom': '20px',
                                   'fontWeight': '900', 'fontSize': '1.4rem', 'letterSpacing': '2px',
                                   'fontFamily': 'Orbitron, monospace',
                                   'textShadow': f'0 0 15px {COLORS["accent"]}, 0 0 30px {COLORS["accent"]}40',
                                   'textTransform': 'uppercase'
                               }),
                        dcc.Graph(id='chart-4', 
                                 config={'displayModeBar': True, 'displaylogo': False},
                                 style={'height': '800px'})
                    ], style=chart_style)
                    
                ], style={
                    'height': '90vh',  # Set viewport height
                    'overflowY': 'auto',  # Enable vertical scrolling
                    'paddingRight': '15px',  # Space for scrollbar
                    'scrollbarWidth': 'thin',  # Thin scrollbar for Firefox
                    'scrollbarColor': f'{COLORS["neon_blue"]} {COLORS["sidebar_bg"]}'  # Custom scrollbar colors
                })
                
            ], width=8)  # Right side takes 8/12 columns
            
        ], className="g-4")
    ], fluid=True)
], style={
    'backgroundColor': COLORS['dark_bg'], 
    'minHeight': '100vh', 
    'padding': '0',
    'fontFamily': 'Rajdhani, Orbitron, monospace'
})


🎨 Creating main layout to match template...


In [13]:
# === DASHBOARD CALLBACKS (MATCHING TEMPLATE PATTERN) ===
print("🔗 Creating callbacks to match template with your actual data...")

# Main callback to update student data, KPIs, and filtered data
@app.callback(
    [Output('student-data-table', 'children'),
     Output('kpi-cards-left', 'children'),
     Output('filtered-data', 'data')],
    [Input('search-box', 'value'),
     Input('start-date-picker', 'date'),
     Input('end-date-picker', 'date'),
     Input('btn-all-years', 'n_clicks')]
)
def update_dashboard(search_value, start_date, end_date, btn_clicks):
    from dash import callback_context
    
    # Start with full dataset (your actual data)
    filtered_df = df.copy()
    
    # Handle "All Time" button
    if callback_context.triggered and 'btn-all-years' in callback_context.triggered[0]['prop_id']:
        start_date = None
        end_date = None
    
    # Apply ONLY date filters for charts and KPIs (NOT search filter)
    chart_filtered_df = filtered_df.copy()
    if start_date and 'COMMENCEMENT DATE' in chart_filtered_df.columns:
        chart_filtered_df = chart_filtered_df[pd.to_datetime(chart_filtered_df['COMMENCEMENT DATE']) >= pd.to_datetime(start_date)]
    if end_date and 'COMMENCEMENT DATE' in chart_filtered_df.columns:
        chart_filtered_df = chart_filtered_df[pd.to_datetime(chart_filtered_df['COMMENCEMENT DATE']) <= pd.to_datetime(end_date)]
    
    # Calculate KPIs from date-filtered data only
    total_students = len(chart_filtered_df)
    avg_gpa = chart_filtered_df['CUMULATIVE_GPA'].mean() if len(chart_filtered_df) > 0 and 'CUMULATIVE_GPA' in chart_filtered_df.columns else 0
    avg_completion = chart_filtered_df['COMPLETION_MONTHS'].mean() if len(chart_filtered_df) > 0 and 'COMPLETION_MONTHS' in chart_filtered_df.columns else 0
    
    # Pass rate calculation (assuming GPA >= 2.0 is pass)
    if len(chart_filtered_df) > 0 and 'CUMULATIVE_GPA' in chart_filtered_df.columns:
        pass_rate = (chart_filtered_df['CUMULATIVE_GPA'] >= 2.0).mean() * 100
    else:
        pass_rate = 0
    
    # Create KPI display (template style with your metrics)
    kpi_display = html.Div([
        html.Div([
            html.H3(f"{total_students:,}", style={
                'color': COLORS['neon_blue'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Total Students", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{avg_gpa:.2f}" if avg_gpa > 0 else "N/A", style={
                'color': COLORS['neon_green'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Average GPA", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{avg_completion:.1f}" if avg_completion > 0 else "N/A", style={
                'color': COLORS['accent'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Avg Completion (months)", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ]),
        html.Div([
            html.H3(f"{pass_rate:.1f}%" if pass_rate > 0 else "N/A", style={
                'color': COLORS['neon_purple'], 'fontWeight': '900', 'margin': '0',
                'fontFamily': 'Orbitron, monospace', 'fontSize': '2rem'
            }),
            html.P("Pass Rate", style={
                'color': COLORS['text'], 'margin': '5px 0 15px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            })
        ])
    ])
    
    # Create student data table - show first 10 by default, filter by search if provided
    if search_value:
        # Apply search filter only for display
        search_filtered = df[df['STUDENT ID'].astype(str).str.contains(str(search_value), na=False, case=False)]
        display_students = search_filtered.head(10)  # Show top 10 matches
    else:
        # Show first 10 students by default
        display_students = df.head(10)
    
    student_cards = []
    for _, row in display_students.iterrows():
        # Build card with available columns
        card_content = [
            html.P(f"ID: {row['STUDENT ID']}", style={
                'color': COLORS['neon_blue'], 'fontWeight': 'bold', 'margin': '5px 0',
                'fontFamily': 'Orbitron, monospace'
            })
        ]
        
        # Add available information
        if 'GENDER' in row and pd.notna(row['GENDER']):
            card_content.append(html.P(f"Gender: {row['GENDER']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'CITIZENSHIP' in row and pd.notna(row['CITIZENSHIP']):
            card_content.append(html.P(f"Citizenship: {row['CITIZENSHIP']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'CUMULATIVE_GPA' in row and pd.notna(row['CUMULATIVE_GPA']):
            card_content.append(html.P(f"GPA: {row['CUMULATIVE_GPA']:.2f}", style={
                'color': COLORS['neon_green'], 'fontWeight': 'bold', 'margin': '3px 0',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'COURSE FUNDING' in row and pd.notna(row['COURSE FUNDING']):
            card_content.append(html.P(f"Funding: {row['COURSE FUNDING']}", style={
                'color': COLORS['text'], 'margin': '3px 0', 'fontSize': '0.9rem',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        if 'PERFORMANCE_TIER' in row and pd.notna(row['PERFORMANCE_TIER']):
            card_content.append(html.P(f"Performance: {row['PERFORMANCE_TIER']}", style={
                'color': COLORS['accent'], 'fontWeight': 'bold', 'margin': '3px 0',
                'fontFamily': 'Rajdhani, monospace'
            }))
        
        # Add separator
        card_content.append(html.Hr(style={'borderColor': COLORS['text_dim'], 'margin': '10px 0'}))
        
        card = html.Div(card_content, style={
            'padding': '10px', 'margin': '5px 0',
            'border': f'1px solid {COLORS["text_dim"]}40',
            'borderRadius': '10px',
            'backgroundColor': f'{COLORS["card_bg"]}60'
        })
        student_cards.append(card)
    
    student_table = html.Div(student_cards)
    
    # Return data for charts (date-filtered only, no search filter)
    return student_table, kpi_display, chart_filtered_df.to_dict('records')

# Chart callbacks to display your existing figures with dynamic data AND preserve your interactive filters
@app.callback(Output('chart-1', 'figure'), Input('filtered-data', 'data'))
def update_chart_1(filtered_data):
    if not filtered_data:
        return fig1  # Return original with all your filters intact
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY chart with "No Data" message
        fig_empty = px.treemap(
            pd.DataFrame({'x': ['No Data Found'], 'y': [1]}),
            path=['x'],
            values='y',
            title='<b>🚫 No Students Found in Selected Date Range</b><br><sub>Adjust your date filters or click "All Time" to see all data</sub>'
        )
        fig_empty.update_layout(
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            title_font_size=18,
            title_x=0.5,
            title_font_color=COLORS['danger'],
            paper_bgcolor=COLORS['dark_bg'],
            height=700,
            margin=dict(t=150, l=20, r=20, b=20)
        )
        fig_empty.update_traces(
            textfont_color=COLORS['danger'],
            marker_colorscale=[[0, COLORS['danger']], [1, COLORS['danger']]]
        )
        return fig_empty
    
    # Regenerate treemap with filtered data but KEEP the original structure
    treemap_data = filtered_df.groupby(['CITIZENSHIP', 'GENDER', 'COURSE FUNDING']).size().reset_index(name='COUNT')
    treemap_data['PERCENTAGE'] = (treemap_data['COUNT'] / len(filtered_df) * 100).round(1)
    
    fig_updated = px.treemap(
        treemap_data,
        path=['CITIZENSHIP', 'GENDER', 'COURSE FUNDING'],
        values='COUNT',
        title=f'<b>🌍 Student Population Breakdown ({len(filtered_df)} students)</b><br><sub>Distribution by Citizenship, Gender, and Funding Type • Click to drill down</sub>',
        color='COUNT',
        color_continuous_scale='Viridis',
        hover_data={'PERCENTAGE': ':.1f%'}
    )
    
    fig_updated.update_layout(
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        title_font_size=18,
        title_x=0.5,
        title_font_color=COLORS['light'],
        paper_bgcolor=COLORS['dark_bg'],
        height=700,
        margin=dict(t=150, l=20, r=20, b=20)
    )
    
    fig_updated.update_traces(
        textinfo='label+value+percent parent',
        textfont_size=11,
        textfont_color=COLORS['light'],
        hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Percentage: %{customdata[0]:.1f}%<extra></extra>'
    )
    
    return fig_updated

@app.callback(Output('chart-2', 'figure'), Input('filtered-data', 'data'))
def update_chart_2(filtered_data):
    if not filtered_data:
        return fig2  # Return original with ALL your interactive features
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY scatter plot with "No Data" message
        fig_empty = go.Figure()
        fig_empty.add_trace(go.Scatter(
            x=[0], y=[2.5], mode='text',
            text=['No Data Found<br>Adjust date filters or click "All Time"'],
            textfont=dict(size=20, color=COLORS['danger'], family='Orbitron, monospace'),
            showlegend=False
        ))
        fig_empty.update_layout(
            title=dict(
                text='<b>🚫 No Students Found in Selected Date Range</b><br><sub>GPA vs Completion Time Analysis - No data to display</sub>',
                font=dict(size=20, family='Orbitron, monospace', color=COLORS['danger']),
                x=0.5, y=0.95
            ),
            xaxis=dict(
                title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
                tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
                gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[0, 10]
            ),
            yaxis=dict(
                title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
                tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
                gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[1.0, 4.0]
            ),
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            paper_bgcolor=COLORS['dark_bg'], plot_bgcolor='rgba(0,0,0,0)',
            height=700, margin=dict(t=150, l=80, r=120, b=80), showlegend=False
        )
        return fig_empty
    
    # Recreate your FULL interactive scatter plot with ALL original features
    scatter_data = filtered_df.dropna(subset=['CUMULATIVE_GPA', 'COMPLETION_MONTHS'])
    
    if len(scatter_data) == 0:
        return fig2
    
    # Fix None values in SUCCESS_SCORE and RISK_SCORE
    scatter_data = scatter_data.copy()
    if 'SUCCESS_SCORE' not in scatter_data.columns or scatter_data['SUCCESS_SCORE'].isna().all():
        # Create success score if missing or all None
        scatter_data['SUCCESS_SCORE'] = (scatter_data['CUMULATIVE_GPA'] / 4.0 * 50) + \
                                       ((scatter_data['COMPLETION_MONTHS'].max() - scatter_data['COMPLETION_MONTHS']) / 
                                        scatter_data['COMPLETION_MONTHS'].max() * 50)
    else:
        # Fill None values with calculated score
        mask = scatter_data['SUCCESS_SCORE'].isna()
        scatter_data.loc[mask, 'SUCCESS_SCORE'] = (scatter_data.loc[mask, 'CUMULATIVE_GPA'] / 4.0 * 50) + \
                                                 ((scatter_data['COMPLETION_MONTHS'].max() - scatter_data.loc[mask, 'COMPLETION_MONTHS']) / 
                                                  scatter_data['COMPLETION_MONTHS'].max() * 50)
    
    if 'RISK_SCORE' not in scatter_data.columns or scatter_data['RISK_SCORE'].isna().all():
        scatter_data['RISK_SCORE'] = 1 - (scatter_data['SUCCESS_SCORE'] / 100)
    else:
        # Fill None values
        mask = scatter_data['RISK_SCORE'].isna()
        scatter_data.loc[mask, 'RISK_SCORE'] = 1 - (scatter_data.loc[mask, 'SUCCESS_SCORE'] / 100)
        
    if 'RISK_CATEGORY' not in scatter_data.columns:
        scatter_data['RISK_CATEGORY'] = pd.cut(scatter_data['RISK_SCORE'], 
                                             bins=[0, 0.3, 0.7, 1.0], 
                                             labels=['Low Risk', 'Medium Risk', 'High Risk'])
    
    fig_updated = go.Figure()
    
    # Recreate ALL your original traces with citizenship filtering
    citizenship_types = scatter_data['CITIZENSHIP'].unique()
    for i, citizenship in enumerate(citizenship_types):
        citizenship_subset = scatter_data[scatter_data['CITIZENSHIP'] == citizenship]
        
        # Regular traces
        fig_updated.add_trace(go.Scatter(
            x=citizenship_subset['COMPLETION_MONTHS'],
            y=citizenship_subset['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (citizenship_subset['RISK_SCORE'] * 4),
                color=citizenship_subset['SUCCESS_SCORE'],  # Now guaranteed to be numeric
                colorscale='Viridis',
                showscale=True if i == 0 else False,
                colorbar=dict(
                    title="Success<br>Score<br>(0-100)",
                    titlefont=dict(size=12, family='Arial Black'),
                    tickfont=dict(size=10),
                    len=0.8
                ) if i == 0 else None,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f}<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
                  f"Success Score: {row['SUCCESS_SCORE']:.1f}<br>" +
                  f"Risk Category: {row['RISK_CATEGORY']}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in citizenship_subset.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} ({len(citizenship_subset)} students)',
            visible=True
        ))
    
    # Add HIGH ACHIEVERS traces (preserving your logic)
    for i, citizenship in enumerate(citizenship_types):
        high_achievers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                     (scatter_data['CUMULATIVE_GPA'] > 3.5) &
                                     (scatter_data['COMPLETION_MONTHS'] < 5)]
        
        fig_updated.add_trace(go.Scatter(
            x=high_achievers['COMPLETION_MONTHS'],
            y=high_achievers['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (high_achievers['RISK_SCORE'] * 4) if 'RISK_SCORE' in high_achievers.columns else 12,
                color=high_achievers['SUCCESS_SCORE'] if 'SUCCESS_SCORE' in high_achievers.columns else citizenship_colors[i % len(citizenship_colors)],
                colorscale='Viridis',
                showscale=False,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f} ⭐⭐<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months 🚀<br>" +
                  f"Success Score: {row.get('SUCCESS_SCORE', 'N/A')}<br>" +
                  f"Risk Category: {row.get('RISK_CATEGORY', 'N/A')}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in high_achievers.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} High Achievers ({len(high_achievers)} students)',
            visible=False
        ))
    
    # Add UNDERPERFORMERS traces (preserving your logic)
    for i, citizenship in enumerate(citizenship_types):
        underperformers = scatter_data[(scatter_data['CITIZENSHIP'] == citizenship) & 
                                      (scatter_data['CUMULATIVE_GPA'] < 2.5)]
        
        fig_updated.add_trace(go.Scatter(
            x=underperformers['COMPLETION_MONTHS'],
            y=underperformers['CUMULATIVE_GPA'],
            mode='markers',
            marker=dict(
                size=12 + (underperformers['RISK_SCORE'] * 4) if 'RISK_SCORE' in underperformers.columns else 12,
                color=underperformers['SUCCESS_SCORE'] if 'SUCCESS_SCORE' in underperformers.columns else citizenship_colors[i % len(citizenship_colors)],
                colorscale='Viridis',
                showscale=False,
                line=dict(width=2, color='white'),
                opacity=0.8,
                symbol='circle'
            ),
            text=[f"Student ID: {row['STUDENT ID']}<br>" +
                  f"GPA: {row['CUMULATIVE_GPA']:.2f} 🚨<br>" +
                  f"Completion: {row['COMPLETION_MONTHS']:.1f} months<br>" +
                  f"Success Score: {row.get('SUCCESS_SCORE', 'N/A')}<br>" +
                  f"Risk Category: {row.get('RISK_CATEGORY', 'N/A')}<br>" +
                  f"Citizenship: {row['CITIZENSHIP']}<br>" +
                  f"Gender: {row['GENDER']}<br>" +
                  f"Funding: {row['COURSE FUNDING']}"
                  for _, row in underperformers.iterrows()],
            hovertemplate='%{text}<extra></extra>',
            name=f'{citizenship} Underperformers ({len(underperformers)} students)',
            visible=False
        ))
    
    # Recreate ALL your dropdown buttons (preserving your original logic)
    num_citizenship = len(citizenship_types)
    dropdown_buttons = [
        # Show all citizenship types (regular view)
        dict(label='🌍 All Citizenship Types', 
             method='update', 
             args=[{'visible': [True] * num_citizenship + [False] * (num_citizenship * 2)}]),
    ]
    
    # Add individual citizenship options (regular view)
    for i, citizenship in enumerate(citizenship_types):
        visibility = [False] * (num_citizenship * 3)
        visibility[i] = True
        dropdown_buttons.append(
            dict(label=f'🎯 {citizenship} Only',
                 method='update',
                 args=[{'visible': visibility}])
        )
    
    # Add high achievers view
    high_achiever_visibility = [False] * num_citizenship + [True] * num_citizenship + [False] * num_citizenship
    dropdown_buttons.append(
        dict(label='⭐ High Achievers (GPA > 3.5 & < 5 months)', 
             method='update', 
             args=[{'visible': high_achiever_visibility}])
    )
    
    # Add underperformers view
    underperformer_visibility = [False] * (num_citizenship * 2) + [True] * num_citizenship
    dropdown_buttons.append(
        dict(label='🚨 Underperformers (GPA < 2.5)', 
             method='update', 
             args=[{'visible': underperformer_visibility}])
    )
    
    fig_updated.update_layout(
        title=dict(
            text=f'<b>🎯 Interactive GPA vs Completion Time Analysis ({len(filtered_df)} students)</b><br>' +
                 '<sub>Success score by color • Risk level by marker size • Use filters to explore different groups</sub>',
            font=dict(size=20, family='Orbitron, monospace', color=COLORS['light']),
            x=0.5,
            y=0.95
        ),
        xaxis=dict(
            title=dict(text='Completion Time (Months)', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_blue'])),
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[0, max(scatter_data['COMPLETION_MONTHS']) + 5] if len(scatter_data) > 0 else [0, 10]
        ),
        yaxis=dict(
            title=dict(text='Cumulative GPA', font=dict(size=14, family='Orbitron, monospace', color=COLORS['neon_purple'])),
            tickfont=dict(size=12, family='Orbitron, monospace', color=COLORS['text']),
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[1.0, 4.0]
        ),
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=80, r=120, b=80),
        showlegend=True,
        legend=dict(
            orientation='v',
            yanchor='top',
            y=0.98,
            xanchor='left',
            x=1.02,
            bgcolor=COLORS['card_bg'],
            bordercolor=COLORS['neon_blue'],
            borderwidth=1,
            font=dict(color=COLORS['light'])
        ),
        
        # PRESERVE your interactive dropdown menu
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction="down",
                showactive=True,
                x=0.02,
                xanchor="left",
                y=1.12,
                yanchor="top",
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_green'],
                borderwidth=2,
                font=dict(color=COLORS['light'])
            )
        ],
        
        # PRESERVE your filter label
        annotations=[
            dict(
                text="🔍 Filter by Group:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_green'])
            )
        ]
    )
    
    # Add reference lines (preserving your original styling)
    fig_updated.add_hline(y=2.0, line_dash="dot", line_color=COLORS['danger'], line_width=2, opacity=0.8, 
                          annotation_text="Pass Threshold (2.0)")
    fig_updated.add_hline(y=3.5, line_dash="dot", line_color=COLORS['success'], line_width=2, opacity=0.8, 
                          annotation_text="Excellence Threshold (3.5)")
    
    return fig_updated

@app.callback(Output('chart-3', 'figure'), Input('filtered-data', 'data'))
def update_chart_3(filtered_data):
    if not filtered_data:
        return fig3  # Return original with ALL your dropdowns
    
    # Convert filtered data back to DataFrame
    filtered_df = pd.DataFrame(filtered_data)
    
    if len(filtered_df) == 0:
        # Create EMPTY box plot with "No Data" message
        fig_empty = go.Figure()
        fig_empty.add_trace(go.Scatter(
            x=[0], y=[15], mode='text',
            text=['No Data Found<br>Adjust date filters or click "All Time"'],
            textfont=dict(size=20, color=COLORS['danger'], family='Orbitron, monospace'),
            showlegend=False
        ))
        fig_empty.update_layout(
            title='<b>🚫 No Students Found in Selected Date Range</b><br><sub>Completion Time Distribution - No data to display</sub>',
            title_font_size=18, title_x=0.5, title_font_color=COLORS['danger'],
            yaxis_title='Completion Time (Months)', xaxis_title='Citizenship Status',
            font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
            paper_bgcolor=COLORS['dark_bg'], plot_bgcolor='rgba(0,0,0,0)',
            height=700, margin=dict(t=150, l=80, r=80, b=80), showlegend=False,
            yaxis=dict(gridcolor='rgba(200,200,200,0.2)', gridwidth=1, range=[0, 30],
                      title_font_color=COLORS['neon_green'], tickfont=dict(color=COLORS['text'])),
            xaxis=dict(gridcolor='rgba(200,200,200,0.2)', gridwidth=1,
                      title_font_color=COLORS['neon_blue'], tickfont=dict(color=COLORS['text']))
        )
        return fig_empty
    
    # Regenerate box plot with filtered data but PRESERVE all your dropdown functionality
    completion_data = filtered_df.dropna(subset=['COMPLETION_MONTHS'])
    completion_data = completion_data[completion_data['COMPLETION_MONTHS'] > 0]
    completion_data = completion_data[completion_data['COMPLETION_MONTHS'] <= 60]
    
    if len(completion_data) == 0:
        return fig3
    
    fig_updated = go.Figure()
    
    # Add box plot for each citizenship type in filtered data (preserving your structure)
    for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
        citizenship_data = completion_data[completion_data['CITIZENSHIP'] == citizenship]
        
        fig_updated.add_trace(
            go.Box(
                y=citizenship_data['COMPLETION_MONTHS'],
                name=citizenship,
                marker_color=citizenship_colors[i % len(citizenship_colors)],
                boxpoints='outliers',
                jitter=0.3,
                pointpos=-1.8,
                marker=dict(
                    size=4,
                    opacity=0.6,
                    line=dict(width=1, color='white')
                ),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                             'Completion Time: %{y:.1f} months<br>' +
                             'Student Count: ' + str(len(citizenship_data)) + '<br>' +
                             '<extra></extra>',
                visible=True
            )
        )
    
    # Recreate YOUR dropdown buttons (preserving original logic)
    all_visible = [True] * len(completion_data['CITIZENSHIP'].unique())
    dropdown_buttons = [
        dict(label='All Citizenship Types',
             method='update',
             args=[{'visible': all_visible}])
    ]
    
    # Add individual citizenship type options (preserving your logic)
    for i, citizenship in enumerate(completion_data['CITIZENSHIP'].unique()):
        visibility = [False] * len(completion_data['CITIZENSHIP'].unique())
        visibility[i] = True
        dropdown_buttons.append(
            dict(label=f'{citizenship} Only',
                 method='update',
                 args=[{'visible': visibility}])
        )
    
    fig_updated.update_layout(
        title=f'<b>🔍 Course Completion Time Distribution ({len(filtered_df)} students)</b><br><sub>Interactive box plot analysis of completion duration by citizenship status</sub>',
        title_font_size=18,
        title_x=0.5,
        title_font_color=COLORS['light'],
        yaxis_title='Completion Time (Months)',
        xaxis_title='Citizenship Status',
        font=dict(family='Orbitron, monospace', size=12, color=COLORS['light']),
        paper_bgcolor=COLORS['dark_bg'],
        plot_bgcolor='rgba(0,0,0,0)',
        height=700,
        margin=dict(t=150, l=80, r=80, b=80),
        showlegend=False,
        
        # PRESERVE your dropdown functionality
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction='down',
                showactive=True,
                x=0.02,
                xanchor='left',
                y=1.12,
                yanchor='top',
                bgcolor=COLORS['card_bg'],
                bordercolor=COLORS['neon_purple'],
                borderwidth=2,
                font=dict(color=COLORS['light'])
            )
        ],
        annotations=[
            dict(
                text="🔍 Filter Options:",
                x=0.02, y=1.18,
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(size=12, family='Orbitron, monospace', color=COLORS['neon_purple'])
            )
        ],
        yaxis=dict(
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            range=[0, 60],
            title_font_color=COLORS['neon_green'],
            tickfont=dict(color=COLORS['text'])
        ),
        xaxis=dict(
            gridcolor='rgba(200,200,200,0.2)',
            gridwidth=1,
            title_font_color=COLORS['neon_blue'],
            tickfont=dict(color=COLORS['text'])
        )
    )
    
    return fig_updated

@app.callback(Output('chart-4', 'figure'), Input('filtered-data', 'data'))
def update_chart_4(filtered_data):
    # For the animated chart, preserve it exactly as you made it
    if not filtered_data:
        return fig4
    
    filtered_df = pd.DataFrame(filtered_data)
    total_filtered = len(filtered_df)
    total_original = len(df)
    
    # Clone the original figure and update title to show filter status
    fig_updated = go.Figure(fig4)
    fig_updated.update_layout(
        title=f'<b>📈 Temporal Trends ({total_filtered}/{total_original} students)</b><br><sub>Animated progression over academic periods</sub>'
    )
    
    return fig_updated

# Live time update callback
@app.callback(
    Output('live-time', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_time(n):
    from datetime import datetime
    return datetime.now().strftime('%H:%M:%S')

# "All Time" button callback to reset date pickers to original data range
@app.callback(
    [Output('start-date-picker', 'date'),
     Output('end-date-picker', 'date')],
    Input('btn-all-years', 'n_clicks'),
    prevent_initial_call=True
)
def reset_date_filters(n_clicks):
    if n_clicks:
        # Reset to the actual data range in your dataset
        return datetime(2022, 4, 18), datetime(2025, 4, 24)
    return datetime(2022, 4, 1), datetime(2025, 4, 24)

print("✅ All callbacks configured with your actual data structure!")
print(f"📊 Dashboard ready for {len(df)} students with real metrics!")
print("🔄 Charts will now show EMPTY when no data matches filters!")
print("⏰ 'All Time' button will reset date range automatically!")

🔗 Creating callbacks to match template with your actual data...
✅ All callbacks configured with your actual data structure!
📊 Dashboard ready for 285 students with real metrics!
🔄 Charts will now show EMPTY when no data matches filters!
⏰ 'All Time' button will reset date range automatically!


In [14]:
# === LAUNCH DASHBOARD ===
if __name__ == '__main__':
    
    print("🌟 Dashboard will be available at: http://127.0.0.1:8052/")
   
    
    app.run(debug=True, port=8052)

🌟 Dashboard will be available at: http://127.0.0.1:8052/
